<h1 style="text-align:center">ML model - Binary Classification with transfer learning</h1>
<h2 style="text-align:center"> Geometric shapes recognition</h2>

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
import os, pathlib
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

ImportError: numpy.core._multiarray_umath failed to import

ImportError: numpy.core.umath failed to import

In [ ]:
training_dir = pathlib.Path("Dataset") #path relativo alla directory di training

In [ ]:
training_set, val_set = image_dataset_from_directory(  #metodo di Keras per il caricamento di dataset di immagini
    training_dir, #directory del dataset,
    labels='inferred', #modalità con cui le labels sono ottenute (inferred = mutuate dalla cartella di appartenenza)
    class_names= ["Square", "Triangle", "Empty"], #forziamo "muffin" come label 0 e "chihuahua" come label 1
    label_mode='int', #modalità di labeling (binary perché si sta trattando una classificazione binaria)
    color_mode='rgb', #Scegliamo di utilizzare tutti e 3 i canali dell'immagine
    image_size=(200, 200), #Ridimensioniamo a 300px*300px
    batch_size = 32, #Scegliamo la dimensione del batch, generalmente si scelgono valori compresi fra 20 e 32
    shuffle = True, #Il dataset subirà uno shuffling randomico per evitare che i batch contengano dati polarizzati
    seed = 41, #Seed per lo shuffling randomico
    validation_split= 0.2, #Percentuale di dataset da assegnare al validation set
    subset= "both", #Scegliamo di farci restituire sia il training set che il validation set, ottenuto da una 
                    #segmentazione del primo
    )

Found 15961 files belonging to 3 classes.
Using 12769 files for training.
Using 3192 files for validation.


2024-03-19 12:50:00.990127: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-19 12:50:00.991192: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
base_model = keras.applications.VGG16(
                include_top=False,         # Rimozione dei layers fully connected
                weights="imagenet",        # Utilizzo dei pesi originali
                input_shape=(200,200,3),   # Adattamento dell'input al nostro task
                )

base_model.trainable = False               # Freezo i pesi della VGG16

inputs = layers.Input(shape=(200,200,3))   # Definisco il modello con la functional API
rescale = layers.Rescaling(1./255)(inputs)
base_model = base_model(rescale, training = False)
flattening = layers.Flatten()(base_model)
flat_layer = layers.Dense(32, activation = "relu")(flattening)
flat_layer = layers.Dense(16, activation = "relu")(flat_layer)
output = layers.Dense(3, activation = "softmax")(flat_layer)

TFModel = keras.Model(inputs = inputs, outputs = output)
TFModel.summary() #riepilogo del modello

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 10s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 6, 6, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 18432)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │       589,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,305,123 (58.38 MB)

 Trainable params: 590,435 (2.25 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [ ]:
TFModel.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                      loss="sparse_categorical_crossentropy",
                      metrics=["accuracy"])

In [ ]:
TFhistory = TFModel.fit(training_set,
                  epochs=1,
                  validation_data = val_set,
                  shuffle = True
                  )

2024-03-19 12:50:20.101061: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 327680000 exceeds 10% of free system memory.
2024-03-19 12:50:20.359204: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 327680000 exceeds 10% of free system memory.


  1/400 ━━━━━━━━━━━━━━━━━━━━ 54:18 8s/step - accuracy: 0.2812 - loss: 1.1030

2024-03-19 12:50:25.968604: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 327680000 exceeds 10% of free system memory.
2024-03-19 12:50:26.202059: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 327680000 exceeds 10% of free system memory.


  2/400 ━━━━━━━━━━━━━━━━━━━━ 37:57 6s/step - accuracy: 0.3047 - loss: 1.0849

2024-03-19 12:50:31.689206: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 327680000 exceeds 10% of free system memory.


 11/400 ━━━━━━━━━━━━━━━━━━━━ 43:47 7s/step - accuracy: 0.3547 - loss: 1.0280

In [ ]:
TFModel.save('MLModel.keras')